In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout,GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
TRAIN_DATA_DIR = '/content/tl-train'
VALIDATION_DATA_DIR = '/content/tl-val'
TRAIN_SAMPLES = 143
VALIDATION_SAMPLES = 67
NUM_CLASSES = 3
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_directory(
                        TRAIN_DATA_DIR,
                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        seed=12345,
                        class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
                        VALIDATION_DATA_DIR,
                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                        batch_size=BATCH_SIZE,
                        shuffle=False,
                        class_mode='categorical')

Found 141 images belonging to 3 classes.
Found 67 images belonging to 3 classes.


In [ ]:
def model_maker():
    base_model = MobileNet(include_top=False, input_shape=(IMG_WIDTH,IMG_HEIGHT,3))
    for layer in base_model.layers[:]:
        layer.trainable = False # Freeze the layers
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [ ]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              # optimizer= tf.train.Adam(lr=0.001),
              optimizer=tf.optimizers.Adam(lr=0.001),
              metrics=['acc'])
num_steps = math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE)              
model.fit_generator(train_generator,
                    steps_per_epoch = num_steps,
                    epochs=10,
                    validation_data = validation_generator,
                    validation_steps = num_steps)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
3/3 [==============================] - 9s 3s/step - loss: 1.0023 - acc: 0.6312 - val_loss: 1.4222 - val_acc: 0.4478
Epoch 2/10
3/3 [==============================] - 3s 986ms/step - loss: 0.7205 - acc: 0.7801
Epoch 3/10
3/3 [==============================] - 6s 2s/step - loss: 0.5288 - acc: 0.7376
Epoch 4/10
3/3 [==============================] - 4s 1s/step - loss: 0.4869 - acc: 0.7518
Epoch 5/10
3/3 [==============================] - 4s 1s/step - loss: 0.5090 - acc: 0.7518
Epoch 6/10
3/3 [==============================] - 6s 2s/step - loss: 0.4263 - acc: 0.7872
Epoch 7/10
3/3 [==============================] - 4s 1s/step - loss: 0.3948 - acc: 0.8369
Epoch 8/10
3/3 [==============================] - 6s 2s/step - loss: 0.3416 - acc: 0.8227
Epoch 9/10
3/3 [==============================] - 3s 994ms/step - loss: 0.3496 - acc: 0.8369
Epoch 10/10
3/3 [==============================] - 3s 998ms/step - loss

In [ ]:
model.save('model.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('model.h5')

In [ ]:
img_path = '/content/tl-val/no/44no.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224,224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array) # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

[[5.3514981e-05 4.1566488e-01 5.8428162e-01]]
{'.ipynb_checkpoints': 0, 'no': 1, 'yes': 2}
